In [1]:
import requests
import json
import numpy as np
import itertools

from pymatgen.phasediagram.pdanalyzer import PDAnalyzer
from pymatgen.matproj.rest import MPRester 
from pymatgen.phasediagram.pdmaker import PhaseDiagram
from pymatgen.core.periodic_table import Element
from pymatgen.core.composition import Composition


API_key = 'NrBIvm9wt7Hq1fSD' # Get API key from https://materialsproject.org/open
mp = MPRester(API_key)


In [2]:
### Get all stable Ternary systems in the database
data = {
    'criteria': {"nelements" : 3,"e_above_hull" :0}, # 0 energy above hull (stability measure)
    'properties': ['chemsys']
}
r = requests.post('https://materialsproject.org/rest/v2/query',
                 headers={'X-API-KEY': API_key},
                 data={k: json.dumps(v) for k,v in data.iteritems()})
response = r.json() # a dict of all stable ternary systems (need to check for duplicates)

ternary_sys = []
for e in response['response']:
    ternary_sys.append(e['chemsys'].split('-'))
# remove duplicates
ternary_sys.sort()
ternary_sys = list(ternary_sys for ternary_sys,_ in itertools.groupby(ternary_sys)) 

for i in range(0,len(ternary_sys)):
    ternary_sys[i] = '-'.join(ternary_sys[i])
len(ternary_sys)

8573

In [3]:
def prop():
    p = np.arange(0.05,1,0.05)
    p = np.around(p,decimals=2)
    p = p.tolist()
    return p

In [4]:
%doctest_mode
from numba import jit

def material_load(d, sep='-', p = prop()):
    return_data = []
     
    d = d.split('-')
    
    # Create a phase diagram object for the following system:
    entry = mp.get_entries_in_chemsys([d[0],d[1]]) # gets the entries of the chemical system
    pd = PhaseDiagram(entry) # creates a phasediagram object
    pd_analyse = PDAnalyzer(pd) # creates a phase Diagram analysis object

    # Get the features for various proportions Using the get_hull_energy method;
    # (Need to add documentation)
    for i in range(0,len(p)):
        temp_data = {}
        prop_a = p[i]
        prop_b = p[-(i+1)]
        a = Element(d[0])
        b = Element(d[1])
        try :
            temp_data['system'] = d[0]+'-'+d[1]
            temp_data['A'] = d[0]
            temp_data['B'] = d[1]
            temp_data[d[0]+'_prop'] = prop_a
            temp_data[d[1]+'_prop'] = prop_b
            temp_data['formation_energy'] = pd_analyse.get_hull_energy(Composition.from_dict({d[0]: prop_a, d[1] : prop_b}))

            # Element Property extraction

            temp_data['avg_atomic_mass'] = prop_a*a.atomic_mass + prop_b*b.atomic_mass
            temp_data['avg_row'] = prop_a*a.row + prop_b*b.row
            temp_data['max_z_diff'] = abs (a.Z - b.Z) # Max Difference in atomic number
            temp_data['avg_z'] = prop_a*a.Z + prop_b*b.Z
            temp_data['max_radius_diff'] = abs (a.atomic_radius - b.atomic_radius) # Max Difference in atomic radius
            temp_data['avg_radius'] = prop_a*a.atomic_radius + prop_b*b.atomic_radius
            temp_data['max_en_diff'] = abs (a.X - b.X) # Max Difference in electronegativity
            temp_data['avg_en'] = prop_a*a.X + prop_b*b.X # Avg Difference in electronegativity
            return_data.append(temp_data)
        except TypeError:
            pass
    return return_data

Exception reporting mode: Context
Doctest mode is: OFF


In [ ]:
%doctest_mode
%time



import pandas as pd

master = pd.DataFrame()

for i,item in enumerate(binary_sys, 1):
    item = item.encode('ascii')
    temp = pd.DataFrame(material_load(item))
    master = master.append(temp)
    if i%10 == 0 : print i


In [13]:
master.to_csv('data_binary.csv')

In [ ]:
## Scratch work 
%doctest_mode 
k = Element('Fe')
k.common_oxidation_states
entry = mp.get_entries_in_chemsys(['Ag','As'])
pd = PhaseDiagram(entry)
pd_analyse = PDAnalyzer(pd)
pd_analyse.get_hull_energy(Composition.from_dict({'Ag': 0.02, 'As': 0.98}))
binary_sys[10]
Composition.from_dict({'Fe': 0.02, 'O': 0.98})
i = 1
prop()[-i]

In [ ]:
def material_load_ternary(d, sep='-'):
    return_data = []
     
    d = d.split('-')
    
    if len(d) > 2: # Check For ternary Systems
        entry = mp.get_entries_in_chemsys([d[0],d[1],d[2]]) # gets the entries of the chemical system
        pd = PhaseDiagram(entry) # creates a phasediagram object
        pd_analyse = PDAnalyzer(pd) # creates a phase Diagram analysis object
        
        
        
    else :  
        # Create a phase diagram object for the following system:
        entry = mp.get_entries_in_chemsys([d[0],d[1]]) # gets the entries of the chemical system
        pd = PhaseDiagram(entry) # creates a phasediagram object
        pd_analyse = PDAnalyzer(pd) # creates a phase Diagram analysis object

        # Get the features for various proportions Using the get_hull_energy method;
        # (Need to add documentation)
        for i in range(0,len(p)):
            temp_data = {}
            prop_a = p[i]
            prop_b = p[-(i+1)]
            a = Element(d[0])
            b = Element(d[1])
            try :
                temp_data['system'] = d[0]+'-'+d[1]
                temp_data['A'] = d[0]
                temp_data['B'] = d[1]
                temp_data[d[0]+'_prop'] = prop_a
                temp_data[d[1]+'_prop'] = prop_b
                temp_data['formation_energy'] = pd_analyse.get_hull_energy(Composition.from_dict({d[0]: prop_a, d[1] : prop_b}))

                # Element Property extraction

                temp_data['avg_atomic_mass'] = prop_a*a.atomic_mass + prop_b*b.atomic_mass
                temp_data['avg_row'] = prop_a*a.row + prop_b*b.row
                temp_data['max_z_diff'] = abs (a.Z - b.Z) # Max Difference in atomic number
                temp_data['avg_z'] = prop_a*a.Z + prop_b*b.Z
                temp_data['max_radius_diff'] = abs (a.atomic_radius - b.atomic_radius) # Max Difference in atomic radius
                temp_data['avg_radius'] = prop_a*a.atomic_radius + prop_b*b.atomic_radius
                temp_data['max_en_diff'] = abs (a.X - b.X) # Max Difference in electronegativity
                temp_data['avg_en'] = prop_a*a.X + prop_b*b.X # Avg Difference in electronegativity
                return_data.append(temp_data)
            except TypeError:
                pass
    return return_data

In [20]:
mp.get_data('Fe-O-Ac')

[{u'icsd_ids': [], u'energy': -36.97250668, u'unit_cell_formula': {u'Ac': 1.0, u'Fe': 1.0, u'O': 3.0}, u'oxide_type': u'oxide', u'hubbards': {u'Ac': 0.0, u'Fe': 5.3, u'O': 0.0}, u'task_ids': [u'mp-864241', u'mp-865551', u'mp-866233', u'mp-866366'], u'band_gap': 0.9887999999999995, u'e_above_hull': 0, u'nsites': 5, u'icsd_id': [], u'elements': [u'Ac', u'Fe', u'O'], u'tags': None, u'pretty_formula': u'AcFeO3', u'volume': 64.69142099382053, u'total_magnetization': 5.0000009, u'is_hubbard': True, u'formation_energy_per_atom': -2.88781133025, u'cif': u"#\\#CIF1.1\n##########################################################################\n#               Crystallographic Information Format file \n#               Produced by PyCifRW module\n# \n#  This is a CIF file.  CIF has been adopted by the International\n#  Union of Crystallography as the standard for data archiving and \n#  transmission.\n#\n#  For information on this file format, follow the CIF links at\n#  http://www.iucr.org\n#####

In [16]:
mp.get_structures('Fe-O-Ac')

[Structure Summary
Lattice
    abc : 4.0143531500000273 4.0143480100010169 4.0143579600010177
 angles : 90.000081069110308 89.999993149097477 89.999988581811039
 volume : 64.691420993820529
      A : 4.0143531499999998 3.9999999999999998e-07 2.4999999999999999e-07
      B : 3.9999999999999998e-07 4.01434801 -2.83e-06
      C : 2.2999999999999999e-07 -2.8499999999999998e-06 4.0143579599999999
PeriodicSite: Ac (4.0144, -0.0000, 4.0144) [1.0000, 0.0000, 1.0000]
PeriodicSite: Fe (2.0072, 2.0072, 2.0072) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (2.0072, 2.0072, 4.0144) [0.5000, 0.5000, 1.0000]
PeriodicSite: O (2.0072, 4.0143, 2.0072) [0.5000, 1.0000, 0.5000]
PeriodicSite: O (4.0143, 2.0072, 2.0072) [1.0000, 0.5000, 0.5000]]

In [5]:
data = mp.query(criteria={}, properties=["task_id"])

In [28]:
import pandas as pd
import  itertools
elements_test = pd.read_table('elements.txt', delimiter="\t") # elements data
elements_test.index = elements_test.symbol # Convert the index as the elements symbols 

binary_sys_test = []
element_list_test = list(elements_test.symbol)
for k in itertools.combinations(element_list_test,2):
     binary_sys_test.append(k[0]+'-'+k[1])
binary_sys_test = sorted (binary_sys_test)    
len(binary_sys_test)  #4371 systems; 94 C 2 combination of 94 known elements

6216

In [10]:
# Dict of all Binary systems
data = {
    'criteria': {"nelements" : 1},
    'properties': ['elements']
}
r = requests.post('https://materialsproject.org/rest/v2/query',
                 headers={'X-API-KEY': API_key},
                 data={k: json.dumps(v) for k,v in data.iteritems()})
elements = r.json()

# Create a list of all binary systems.
import itertools
element_list = []
binary_sys = []
for i in range(0 , len (elements['response'])):
    element_list.append(elements['response'][i]['elements'][0])
element_list = list(set(element_list))
for subset in itertools.combinations(element_list,2):
     binary_sys.append(subset[0]+'-'+subset[1])
binary_sys = sorted (binary_sys)  

len(binary_sys) # 3916 systems; combination of 89 known elements

3916

In [23]:
binary_sys = [x.encode('UTF8') for x in binary_sys]
sorted(list(set(binary_sys_test) - set (binary_sys)))

['Ac-At',
 'Ac-B',
 'Ac-Ba',
 'Ac-Be',
 'Ac-Bi',
 'Ac-Br',
 'Ac-C',
 'Ac-Ca',
 'Ac-Cd',
 'Ac-Ce',
 'Ac-Cl',
 'Ac-Co',
 'Ac-Cr',
 'Ac-Cs',
 'Ac-Cu',
 'Ac-Dy',
 'Ac-Er',
 'Ac-Eu',
 'Ac-F',
 'Ac-Fe',
 'Ac-Fr',
 'Ac-Ga',
 'Ac-Gd',
 'Ac-Ge',
 'Ac-H',
 'Ac-He',
 'Ac-Hf',
 'Ac-Hg',
 'Ac-Ho',
 'Ac-I',
 'Ac-K',
 'Ac-Kr',
 'Ac-La',
 'Ac-Li',
 'Ac-Lu',
 'Ac-Mg',
 'Ac-Mn',
 'Ac-N',
 'Ac-Na',
 'Ac-Nb',
 'Ac-Nd',
 'Ac-Ne',
 'Ac-Ni',
 'Ac-Np',
 'Ac-O',
 'Ac-Os',
 'Ac-P',
 'Ac-Pa',
 'Ac-Pb',
 'Ac-Pd',
 'Ac-Pm',
 'Ac-Po',
 'Ac-Pr',
 'Ac-Pt',
 'Ac-Pu',
 'Ac-Ra',
 'Ac-Rb',
 'Ac-Re',
 'Ac-Rh',
 'Ac-Rn',
 'Ac-Ru',
 'Ac-S',
 'Ac-Sb',
 'Ac-Sc',
 'Ac-Se',
 'Ac-Si',
 'Ac-Sm',
 'Ac-Sn',
 'Ac-Sr',
 'Ac-Ta',
 'Ac-Tb',
 'Ac-Tc',
 'Ac-Te',
 'Ac-Th',
 'Ac-Ti',
 'Ac-Tl',
 'Ac-Tm',
 'Ac-U',
 'Ac-V',
 'Ac-W',
 'Ac-Xe',
 'Ac-Y',
 'Ac-Yb',
 'Ac-Zn',
 'Ac-Zr',
 'Ag-At',
 'Ag-B',
 'Ag-Ba',
 'Ag-Be',
 'Ag-Bi',
 'Ag-Br',
 'Ag-C',
 'Ag-Ca',
 'Ag-Cd',
 'Ag-Ce',
 'Ag-Cl',
 'Ag-Co',
 'Ag-Cr',
 'Ag-Cs',
 'Ag-Cu',
 'Ag-Dy',
 'Ag-E

In [31]:
set(element_list_test) - set(element_list)

{'Am',
 'At',
 'Bh',
 'Bk',
 'Cf',
 'Cm',
 'Cn',
 'Db',
 'Ds',
 'Es',
 'Fm',
 'Fr',
 'Hs',
 'Lr',
 'Md',
 'Mt',
 'No',
 'Po',
 'Ra',
 'Rf',
 'Rg',
 'Rn',
 'Sg'}

In [25]:
binary_sys_test

['Ac-Ag',
 'Ac-Al',
 'Ac-Ar',
 'Ac-As',
 'Ac-At',
 'Ac-Au',
 'Ac-B',
 'Ac-Ba',
 'Ac-Be',
 'Ac-Bi',
 'Ac-Br',
 'Ac-C',
 'Ac-Ca',
 'Ac-Cd',
 'Ac-Ce',
 'Ac-Cl',
 'Ac-Co',
 'Ac-Cr',
 'Ac-Cs',
 'Ac-Cu',
 'Ac-Dy',
 'Ac-Er',
 'Ac-Eu',
 'Ac-F',
 'Ac-Fe',
 'Ac-Fr',
 'Ac-Ga',
 'Ac-Gd',
 'Ac-Ge',
 'Ac-H',
 'Ac-He',
 'Ac-Hf',
 'Ac-Hg',
 'Ac-Ho',
 'Ac-I',
 'Ac-In',
 'Ac-Ir',
 'Ac-K',
 'Ac-Kr',
 'Ac-La',
 'Ac-Li',
 'Ac-Lu',
 'Ac-Mg',
 'Ac-Mn',
 'Ac-Mo',
 'Ac-N',
 'Ac-Na',
 'Ac-Nb',
 'Ac-Nd',
 'Ac-Ne',
 'Ac-Ni',
 'Ac-Np',
 'Ac-O',
 'Ac-Os',
 'Ac-P',
 'Ac-Pa',
 'Ac-Pb',
 'Ac-Pd',
 'Ac-Pm',
 'Ac-Po',
 'Ac-Pr',
 'Ac-Pt',
 'Ac-Pu',
 'Ac-Ra',
 'Ac-Rb',
 'Ac-Re',
 'Ac-Rh',
 'Ac-Rn',
 'Ac-Ru',
 'Ac-S',
 'Ac-Sb',
 'Ac-Sc',
 'Ac-Se',
 'Ac-Si',
 'Ac-Sm',
 'Ac-Sn',
 'Ac-Sr',
 'Ac-Ta',
 'Ac-Tb',
 'Ac-Tc',
 'Ac-Te',
 'Ac-Th',
 'Ac-Ti',
 'Ac-Tl',
 'Ac-Tm',
 'Ac-U',
 'Ac-V',
 'Ac-W',
 'Ac-Xe',
 'Ac-Y',
 'Ac-Yb',
 'Ac-Zn',
 'Ac-Zr',
 'Ag-Al',
 'Ag-Ar',
 'Ag-As',
 'Ag-At',
 'Ag-Au',
 'Ag-B',
 'Ag-Ba',
 'Ag-Be',
 'Ag-

In [32]:
formation_energies = pd.read_table('formation_energies.txt',delimiter='\t' )

In [38]:
is_ternary = formation_energies['composition_id'].str.split().apply(len) == 3
is_stable = formation_energies['stability'] <= 0

ternary_systems = formation_energies[is_stable & is_ternary]
ternary_systems = ternary_systems[['composition_id','stability','delta_e']]

ternary_systems.head()

,composition_id,stability,delta_e
1057,Li1 N1 Zn1,-0.091413,-0.346888
1076,Bi1 Ho1 Pt1,-0.178451,-1.046258
1088,Bi1 Gd1 Pt1,-0.197731,-1.052662
1091,As1 Li1 Mg1,-0.093365,-0.755614
1092,Pd1 Sb1 Sc1,-0.180729,-1.047253
